In [1]:
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import torch
from data_process import load_mat
from sklearn.metrics import accuracy_score
import numpy as np

(40596, 204)
(13533, 204)
(40596, 1)
(13533, 1)


In [2]:
class FullyConnectedNet(torch.nn.Module):
    def __init__(self, n_feature, n_output):
        super(FullyConnectedNet, self).__init__()
        self.hidden_size_1 = 512
        self.hidden_size_2 = 320
        self.hidden_size_3 = 128

        self.hidden_1 = nn.Linear(n_feature, self.hidden_size_1)
        self.hidden_2 = nn.Linear(self.hidden_size_1, self.hidden_size_2)
        self.hidden_3 = nn.Linear(self.hidden_size_2, self.hidden_size_3)
        self.out = nn.Linear(self.hidden_size_3, n_output)

    def forward(self, x):
        x = F.relu(self.hidden_1(x))
        x = F.relu(self.hidden_2(x))
        x = F.relu(self.hidden_3(x))
        x = self.out(x)
        return x

In [3]:
data_path = './data2/Salinas_corrected.mat'
label_path = './data2/Salinas_gt.mat'
train_x, test_x, train_y, test_y = load_mat(data_path, label_path, 0.25)

In [4]:
train_x = Variable(torch.FloatTensor(train_x)).cuda()

In [5]:
test_x = Variable(torch.FloatTensor(test_x)).cuda()
train_y = Variable(torch.LongTensor(np.squeeze(np.asarray(train_y, dtype=np.int64)))).cuda()
test_y = Variable(torch.LongTensor(np.squeeze(np.asarray(test_y, dtype=np.int64))))

In [6]:
fcnn = FullyConnectedNet(n_feature=204, n_output=16)

fcnn.cuda()

EPOCH = 500
BATCH_SIZE = 200
LR = 0.001

optimizer = torch.optim.Adam(fcnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [7]:
for t in range(EPOCH):
    prediction = fcnn(train_x)
    loss = loss_func(prediction, train_y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('epoch {}, train loss: {}'.format(t, loss.data[0]))

epoch 0, train loss: 2.770817279815674
epoch 1, train loss: 2.664440393447876
epoch 2, train loss: 2.553398609161377
epoch 3, train loss: 2.41252064704895
epoch 4, train loss: 2.2373299598693848
epoch 5, train loss: 2.038421630859375
epoch 6, train loss: 1.8411697149276733
epoch 7, train loss: 1.6762698888778687
epoch 8, train loss: 1.5450658798217773
epoch 9, train loss: 1.42690110206604
epoch 10, train loss: 1.324112892150879
epoch 11, train loss: 1.2323700189590454
epoch 12, train loss: 1.1575027704238892
epoch 13, train loss: 1.099145770072937
epoch 14, train loss: 1.047837257385254
epoch 15, train loss: 0.9942001104354858
epoch 16, train loss: 0.964367151260376
epoch 17, train loss: 0.9293386936187744
epoch 18, train loss: 0.8928641676902771
epoch 19, train loss: 0.8692101836204529
epoch 20, train loss: 0.8375950455665588
epoch 21, train loss: 0.8113358020782471
epoch 22, train loss: 0.7872058153152466
epoch 23, train loss: 0.7648479342460632
epoch 24, train loss: 0.74731105566024

epoch 199, train loss: 0.20941980183124542
epoch 200, train loss: 0.21034808456897736
epoch 201, train loss: 0.21212828159332275
epoch 202, train loss: 0.21368472278118134
epoch 203, train loss: 0.21165160834789276
epoch 204, train loss: 0.2096174657344818
epoch 205, train loss: 0.20758147537708282
epoch 206, train loss: 0.2065451294183731
epoch 207, train loss: 0.2063063383102417
epoch 208, train loss: 0.20573829114437103
epoch 209, train loss: 0.20616267621517181
epoch 210, train loss: 0.20715080201625824
epoch 211, train loss: 0.2083965539932251
epoch 212, train loss: 0.20834706723690033
epoch 213, train loss: 0.2081274837255478
epoch 214, train loss: 0.20586179196834564
epoch 215, train loss: 0.20395039021968842
epoch 216, train loss: 0.20130416750907898
epoch 217, train loss: 0.2001170665025711
epoch 218, train loss: 0.20067468285560608
epoch 219, train loss: 0.20186658203601837
epoch 220, train loss: 0.20349161326885223
epoch 221, train loss: 0.20358102023601532
epoch 222, train 

epoch 394, train loss: 0.1540139764547348
epoch 395, train loss: 0.1568821519613266
epoch 396, train loss: 0.15777020156383514
epoch 397, train loss: 0.1579631119966507
epoch 398, train loss: 0.1542108952999115
epoch 399, train loss: 0.15150469541549683
epoch 400, train loss: 0.15056149661540985
epoch 401, train loss: 0.15122713148593903
epoch 402, train loss: 0.15317261219024658
epoch 403, train loss: 0.15515421330928802
epoch 404, train loss: 0.15734291076660156
epoch 405, train loss: 0.1554403305053711
epoch 406, train loss: 0.15320438146591187
epoch 407, train loss: 0.15061292052268982
epoch 408, train loss: 0.14924205839633942
epoch 409, train loss: 0.14921852946281433
epoch 410, train loss: 0.1503332406282425
epoch 411, train loss: 0.15223905444145203
epoch 412, train loss: 0.15368075668811798
epoch 413, train loss: 0.15509817004203796
epoch 414, train loss: 0.15413662791252136
epoch 415, train loss: 0.1529257595539093
epoch 416, train loss: 0.15037982165813446
epoch 417, train l

In [ ]:
test_pred = torch.max(fcnn(test_x), 1)[1].data.numpy().squeeze()
print(accuracy_score(test_pred, test_y.data.numpy()))